## Notions de base

### Vocabulaire

Conteneur 
: Un conteneur est une sorte de bac à sable (sandbox) qui isole un ensemble de processus du système en utilisant les namespaces du noyau linux et leur associe une carte réseau virtuelle. Un conteneur est l'instance d'une image. Généralement, un conteneur fournit un et un seul service.

Image
: Une image est système de fichier isolé qui contient tout ce qui est nécessaire pour l'exécution d'un conteneur. Elle contient aussi d'autres informations comme des metadonnées, des variables d'environnement ou la commande par défaut à executer. Souvent une image s'appuie sur une autre image à laquelle elle apporte des modificiations.

Docker Daemon
: Le docker daemon est un serveur qui gère les images, les conteneurs, les réseaux et les volumes de stockage. Il peut communiquer avec d'autres docker daemons et offre une API standard.

Docker Client
: Le docker client est un outils en ligne de commande pour envoyer des commandes à un ou plusieurs docker daemons locaux ou distants. 


> Par exemple, le docker client peut demander au docker daemon d'exécuter une image ce qui produit un conteneur. Si l'image n'est pas disponible localement, elle est téléchargée par le docker daemon depuis un registry.

### Installation

Docker peut être installé directement sous linux, il partage alors le noyau de l'hôte. Il peut aussi être installé dans une machine virtuelle Linux (sous Linux, Windows et MacOs) par exemple via [Docker Desktop](https://docs.docker.com/get-docker/).

### Exécuter un conteneur

La commande `docker container run` permet d'executer la commande par défaut d'une image dans un espace isolé. Si l'image n'est pas disponible pour le docker daemon, elle est téléchargée à partir d'un registry (par défaut [Docker Hub](http://hub.docker.com)). Il peut aussi être utilisé à distance depuis le cloud.

Comme premier exemple, exécutons l'image [hello-world](https://hub.docker.com/_/hello-world) qui illustre ce fonctionnement :

In [6]:
docker container run hello-world

Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world

Digest: sha256:6e8b6f026e0b9c419ea0fd02d3905dd0952ad1feea67543f525c73a0a790fefb
Status: Downloaded newer image for hello-world:latest

Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (arm64v8)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 htt

Une commande docker commencent par le type d'objet qu'elle manipule (`container`, `image`, ...) suivi d'une sous-commande. Par exempke, il est possible de précharger ou de mettre à jour une image avec la sous-commande `pull`.

Pour télécharger ou mettre à jour [Alpine Linux](https://www.alpinelinux.org/) qui est une distribution linux très légère :

In [13]:
docker image pull alpine

Using default tag: latest
latest: Pulling from library/alpine
Digest: sha256:69665d02cb32192e52e07644d76bc6f25abeb5410edc1c7a81a10ba3f0efb90a
Status: Image is up to date for alpine:latest
docker.io/library/alpine:latest


Le format du nom d'une image est `[registry_hostname[:port]][path]image_name[:tag]`.

si l'adresse du registry est omise c'est celle de dockerhub qui est utilisé (docker.io). 

Si l'image est "officielle" `path` peut-être omis et vaut `library`, sinon il s'agit généralement du compte de l'utilisateur qui fourni l'image. 

`tag` permet de différencier les versions d'une image, s'il est omis il vaut `latest`. 

Ainsi, le nom d'image `alpine` correspond en fait à `docker.io/library/alpine:latest`. La [documentation](https://hub.docker.com/_/alpine/tags) indique les tags existants.

Il est possible de choisir la commande à utiliser au lancement du container en l'indiquant après le nom de l'image. Des variables d'environnement peuvent êtrée crées dans le conteneur avec l'option `--env`.

In [72]:
docker container run \
    alpine uname -a

Linux 04d572f58590 5.15.49-linuxkit #1 SMP PREEMPT Tue Sep 13 07:51:32 UTC 2022 aarch64 Linux


In [73]:
docker container run --env FIRSTNAME="John" --env NAME="Doe" \
    alpine env

PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
HOSTNAME=24c10da3673b
FIRSTNAME=John
NAME=Doe
HOME=/root


In [20]:
docker container run \
    alpine sh -c "echo 'Hi !' > hello.txt ; cat hello.txt"

Hi !


Si la commande est interactive (comme un shell bash par exemple il faut préciser `--interactive` ou `-it`) pour utiliser l'entrée et la sortie standard. 

Il est aussi possible de donner un nom unique à conteneur qui peut être utilisé à la place de son ID avec l'option `--name`.

In [74]:
(
cat <<EOF
cd /root
ls -al
EOF
) | docker run --interactive --quiet --name my-bash ubuntu bash - 

total 16
drwx------ 2 root root 4096 Mar  1 02:16 .
drwxr-xr-x 1 root root 4096 Mar 10 08:32 ..
-rw-r--r-- 1 root root 3106 Oct 15  2021 .bashrc
-rw-r--r-- 1 root root  161 Jul  9  2019 .profile


In [83]:
docker stop my-redis
docker rm my-redis


docker rmi redis:7.0.9
docker image prune --force

Error response from daemon: No such container: my-redis
Error response from daemon: No such container: my-redis
Error response from daemon: No such image: redis:7.0.9
Total reclaimed space: 0B


L'exécution d'un conteneur peut être faite tâche de fond avec l'option `--detach` ou `-d`.

Par exemple, pour lancer [redis](https://redis.com/), une base de donnée NoSQL de type clé/valeur :

In [85]:
docker run --detach --name my-redis redis:7.0.9

Unable to find image 'redis:7.0.9' locally
7.0.9: Pulling from library/redis

ba0fb1b5: Already exists 
abea72ba: Pulling fs layer 
c855d06e: Pulling fs layer 
636102fd: Pulling fs layer 
9543da32: Pulling fs layer 
Digest: sha256:e50c7e23f79ae81351beacb20e004720d4bed657415e68c2b1a2b5557c075ce0
Status: Downloaded newer image for redis:7.0.9
87f4b196a0754c6b71eb54d92dd74024b91e9626e33803fe1577fec08d182241


Les conteneurs produisent généralement le log du service sur la sortie standard. Quand `--detach` est utilisé la commande `logs` permet de le consulter (ajouter `-f` pour un suivi en continu).

In [25]:
docker logs my-redis

1:C 09 Mar 2023 11:08:25.491 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
1:C 09 Mar 2023 11:08:25.491 # Redis version=7.0.9, bits=64, commit=00000000, modified=0, pid=1, just started
1:C 09 Mar 2023 11:08:25.491 # Warning: no config file specified, using the default config. In order to specify a config file use redis-server /path/to/redis.conf
1:M 09 Mar 2023 11:08:25.491 * monotonic clock: POSIX clock_gettime
1:M 09 Mar 2023 11:08:25.492 * Running mode=standalone, port=6379.
1:M 09 Mar 2023 11:08:25.492 # Server initialized
1:M 09 Mar 2023 11:08:25.493 * Ready to accept connections
1:signal-handler (1678360274) Received SIGTERM scheduling shutdown...
1:M 09 Mar 2023 11:11:14.757 # User requested shutdown...
1:M 09 Mar 2023 11:11:14.757 * Saving the final RDB snapshot before exiting.
1:M 09 Mar 2023 11:11:14.759 * DB saved on disk
1:M 09 Mar 2023 11:11:14.759 # Redis is now ready to exit, bye bye...
1:C 09 Mar 2023 11:11:16.528 # oO0OoO0OoO0Oo Redis is starting oO0OoO0OoO0Oo
1:C 09

La sous-commande `ls` permet d'obtenir tous les conteneurs en cours d'exécution.

In [27]:
docker container ls

CONTAINER ID   IMAGE                  COMMAND                  CREATED        STATUS        PORTS                    NAMES
939f81ab9801   restjpa:latest         "java -jar /myapp.jar"   10 hours ago   Up 10 hours   0.0.0.0:8088->8080/tcp   restjpa-app-1
15799e339c0d   postgres:15.2-alpine   "docker-entrypoint.s…"   10 hours ago   Up 10 hours   5432/tcp                 restjpa-db-1
63a371adc615   sonatype/nexus3        "/opt/sonatype/nexus…"   17 hours ago   Up 17 hours   0.0.0.0:8081->8081/tcp   nexus
ffa36b13f439   redis:7.0.9            "docker-entrypoint.s…"   21 hours ago   Up 21 hours   6379/tcp                 my-redis


La commande `ls -a` affiche aussi ceux qui se sont arrêtés.

In [29]:
docker container ls -a

CONTAINER ID   IMAGE                     COMMAND                  CREATED              STATUS                          PORTS                    NAMES
4b293186f542   alpine                    "sh -c 'echo 'Hi !' …"   About a minute ago   Exited (0) About a minute ago                            intelligent_jemison
9b4520fbaed9   alpine                    "env"                    About a minute ago   Exited (0) About a minute ago                            eloquent_thompson
090327d4720f   alpine                    "uname -a"               About a minute ago   Exited (0) About a minute ago                            trusting_burnell
d517d72e1445   hello-world               "/hello"                 7 minutes ago        Exited (0) 7 minutes ago                                 stoic_grothendieck
939f81ab9801   restjpa:latest            "java -jar /myapp.jar"   10 hours ago         Up 10 hours                     0.0.0.0:8088->8080/tcp   restjpa-app-1
15799e339c0d   postgres:15.2-alpine      "

Un conteneur s'arrête quand sa commande termine. Il peut être arrêté avec la sous-commande `stop`.

In [31]:
docker container stop my-redis

my-redis


Un conteneur arrêté peut être relancé avec les même paramètres avec la commande `start`.

In [33]:
docker container start my-redis

my-redis


Un container arrêté peut être détruit avec la commande `rm` en indiquant son Id ou son nom.

In [35]:
docker rm my-bash

my-bash


La commande `exec` permet d'executer une commande dans un conteneur en fonctionnement. Par exemple, pour utiliser l'interface en ligne de commande de redis depuis le conteneur `my-redis` (qui exécute déjà le serveur) pour ajouter une valeur de clé "a.b@x.fr".

In [37]:
docker exec my-redis redis-cli SET a.b@x.fr "Pierre,Durand,12"

OK


L'option `--rm` de la commande `run` provoque la suppression du conteneur dès l'arrêt.

On peut alors créer un autre conteneur éphémère pour exécuter la commande de récupération d'une valeur de redis via le réseau (la partie réseau est expliquée plus tard) à partir de l'image du serveur. 

In [39]:
docker run --rm --link my-redis redis:7.0.9 redis-cli -h my-redis GET a.b@x.fr

Pierre,Durand,12


In [41]:
docker container ls 

CONTAINER ID   IMAGE                  COMMAND                  CREATED        STATUS              PORTS                    NAMES
939f81ab9801   restjpa:latest         "java -jar /myapp.jar"   10 hours ago   Up 10 hours         0.0.0.0:8088->8080/tcp   restjpa-app-1
15799e339c0d   postgres:15.2-alpine   "docker-entrypoint.s…"   10 hours ago   Up 10 hours         5432/tcp                 restjpa-db-1
63a371adc615   sonatype/nexus3        "/opt/sonatype/nexus…"   17 hours ago   Up 17 hours         0.0.0.0:8081->8081/tcp   nexus
ffa36b13f439   redis:7.0.9            "docker-entrypoint.s…"   21 hours ago   Up About a minute   6379/tcp                 my-redis


In [42]:
docker container stop my-redis

docker container rm my-redis

my-redis
my-redis


Tous les containeurs arrêtés peuvent être supprimés avec la commande `container prune`.

In [44]:
docker container prune --force

Deleted Containers:
4b293186f54222c03ea87db74890f7ab81f332bc63c155e40b29440dcc3f6a62
9b4520fbaed95b83ec5fe8dfe29684a38bd03c49083524ec39c9c8d1479138d7
090327d4720f6cedba602f6de2118aafd3b486243dec03f951131503a0cb5523
d517d72e14457827b7de8036a85f5595831393bcd7815e501e50fbfb08056d0c
547c6b4e77e68a245ae3d05b5291dc71634650a90863b8e13f6712341f65ba4a
b212f9a84f18fb6ff0a1e7fa7252ac6abfb593956455beebef419a93b0418495
83d5991d1a72c2ea28344ae2f04aa6ffc136e2a702a913aa2e3cb0f61bb9f39d
3e4ceaf1c88e17945b695134fc4fb80813aba581450b66c86371cfb942ce7fdb
6668f54ca7655b169a591a226083f3f90cc8e8b6854749c3b7bb7b15b4b2d0a6
03172e24ff77733e2fc0375ff12ec7a7e23b99a3d7e811609e6ade72a632ec3f
cd1b59c575636338f41b936674a101b611b9cef165359c658fc0ac92608bad5a
c10305ffb274cadb7204b1db5bc7bce02770d1f04e814ebd9ffd9e959edfc71a
d3572fbfe588d4075db329c5e83e86b1f66fd7628b13934fc3d2324b3dd69222
9b488ef0d53c24ab2b95c84798546c9f1c97692cd19455dcf67edff32b240521
de389ee139cc16881877e2c52c459712ce1187dcb1dea21fb8a29a5fafdac065
70432

In [46]:
docker container ls -a

CONTAINER ID   IMAGE                  COMMAND                  CREATED        STATUS        PORTS                    NAMES
939f81ab9801   restjpa:latest         "java -jar /myapp.jar"   10 hours ago   Up 10 hours   0.0.0.0:8088->8080/tcp   restjpa-app-1
15799e339c0d   postgres:15.2-alpine   "docker-entrypoint.s…"   10 hours ago   Up 10 hours   5432/tcp                 restjpa-db-1
63a371adc615   sonatype/nexus3        "/opt/sonatype/nexus…"   17 hours ago   Up 17 hours   0.0.0.0:8081->8081/tcp   nexus


### Gérer les images

Il est possible de lister ou supprimer les images disponibles localement avec la commande `image` et les sous-commandes `ls` et `rm`.

In [48]:
docker image ls

REPOSITORY                 TAG               IMAGE ID       CREATED         SIZE
restjpa                    latest            4b8a8b55ddbb   10 hours ago    281MB
<none>                     <none>            1400205ad3f3   10 hours ago    281MB
<none>                     <none>            a4473206e9ae   10 hours ago    281MB
javahello                  scratch           bd4891efcdd4   11 hours ago    262MB
javahello                  wrapper           4febce8b3a54   11 hours ago    262MB
javahello                  cache             ce8587c00465   11 hours ago    262MB
javahello                  mavenimagestage   0987f84ebd57   11 hours ago    262MB
javahello                  mavenimage        dd60669ea7ed   11 hours ago    573MB
<none>                     <none>            f595c6f0dfd7   11 hours ago    9.87MB
<none>                     <none>            d7f95a92d9a1   11 hours ago    9.87MB
<none>                     <none>            749b6079beda   11 hours ago    9.87MB
<none>        

In [49]:
docker image rm hello-world redis:7.0.9 alpine

Untagged: hello-world:latest
Untagged: hello-world@sha256:6e8b6f026e0b9c419ea0fd02d3905dd0952ad1feea67543f525c73a0a790fefb
Deleted: sha256:46331d942d6350436f64e614d75725f6de3bb5c63e266e236e04389820a234c4
Deleted: sha256:efb53921da3394806160641b72a2cbd34ca1a9a8345ac670a85a04ad3d0e3507
Untagged: redis:7.0.9
Untagged: alpine:latest
Untagged: alpine@sha256:69665d02cb32192e52e07644d76bc6f25abeb5410edc1c7a81a10ba3f0efb90a
Deleted: sha256:d74e625d91152966d38fe8a62c60daadb96d4b94c1a366de01fab5f334806239


In [50]:
docker image prune --force

Deleted Images:
deleted: sha256:a4473206e9ae8dee8acf9be5de3c82e40185f0fdb78ca9eaaaf08733ff11f981
deleted: sha256:7b23ac26e3f2cde18a9541049c3fe1daa01e953276604afc97d909d55c798261
deleted: sha256:7672ab4857bfbc7eccc5b4e16349b2ce4518d8d830ee202f376866d788b94923
deleted: sha256:0423dcc18c501ebace218e2b50b530882877b76908add51bf73655de83a28aab
deleted: sha256:b9d642650ab980fcd4eb99f73f8d3c3fb1001472ee918d31ea2a9fd481c4e8fe
deleted: sha256:1400205ad3f399e05c5da65ab65c511fa18a50914f7e62b7c8b6e29802850e6f
deleted: sha256:f208e63cbde91a079b983a848ad08d68bbc56cd9a17c17ddbf03a191877e229b
deleted: sha256:86a536d8ba2a8fa88949bb04cc1751a1049aed244ea1d2dc7a502530ee266280
deleted: sha256:6d2d54d8475324357d654399191612287d5614e945922c75413e57415a53ee1a
deleted: sha256:c4b6373999c0e282ea9357981d9e662bc700aca1b33dbfceab0e51ef5c1e76eb
deleted: sha256:828a10c39ba2017659a0f0d62512ab890350e66da503ba37c54c89935061980a
deleted: sha256:bf040d87ceb6bfc096a39ff97d252073bc160c71ceca80e2f951ed092aec1ee7
deleted: sha

### Volumes

Un conteneur est généralement éphémère et sans état pour pouvoir être arrêtés et relancés sans perte de données. Les fichiers créés devant être persistants doivent donc l'être en dehors du conteneur. Pour cela, on utilise les volumes. 

Un volume peut être un bind volume c'est-à-dire un montage du système de fichier de l'hôte vers un répertoire du conteneur). 
Ils sont définis avec l'option `-v <source>:<destination>` de la commande `run` que l'on peut uiliser plusieurs fois. Le chemin doit être absolu mais peut utiliser la variable d'environement `${PWD}` pour simuler un chemin relatif. 

Dans l'exemple suivant, un premier conteneur crée le fichier `text.txt` dans un volume monté de l'hôte (le répertoire `/tmp/mydata`) dans le conteneur (à l'emplacement `/data`). Le conteneur est ensuite détruit.
Un second conteneur monte le même répertoire de l'hôte (mais dans le nouveau conteneur dans `/databis`) et affiche le contenu.

In [57]:
docker container run --rm -v /tmp/mydata:/data ubuntu bash -c "echo 'hello'>/data/test.txt"

In [58]:
docker container run --rm -v /tmp/mydata:/databis ubuntu cat /databis/test.txt

hello


un volume peut aussi être un volume nommé géré par le docker daemon dans un système de fichiers virtuel de façon transparente pour l'utilisateur. Il n'est alors plus aussi simple de partager des fichiers entre l'hôte et les conteneurs mais il n'y a plus de problème d'UID du propriétaire ni de droits. Il s'agit de la meilleure solution quand le docker daemon est exécuté sur un cluster et non sur une seule machine car alors le système de fichier peut être distribué.

L'exemple suivant exécute une base de données via l'image de [PostgreSQL](https://hub.docker.com/_/postgres) dans un premier conteneur. Le volume nommé `postgres-test-data` est monté dans le répertoire indiqué dans la documentation de l'image pour contenir la base de données (`/var/lib/postgresql/data`). Comme celui-ci est initialement vide, l'image est construite pour créer alors automatiquement une base de données à partir des variables d'environnement fournies. 

In [62]:
docker run -d --rm \
  --name postgres-test \
  --env POSTGRES_PASSWORD=mysecretpassword \
  -v postgres-test-data:/var/lib/postgresql/data \
  postgres:15.2

docker: Error response from daemon: Conflict. The container name "/postgres-test" is already in use by container "1ca417bd0ac5fcce44846ce8b79e55e08ad1b5b26357d0d19b38dacbfc1f2428". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


Il est possible de monter le même volume et un bind volume dans une autre conteneur par exemple pour faire une sauvegarde. En pratique, on préfère `pg_dump`, mais nous l'illustrons ici avec un simple tar. Dans le cas d'une base de données relationnelles l'arrêt du conteneur obligatoire pour garder la cohérence. 

In [65]:
docker stop postgres-test

postgres-test


In [70]:
docker run --rm \
    -v postgres-test-data:/var/lib/postgresql/data \
    -v /tmp/backup:/backup \
    ubuntu:jammy \
    tar zcf /backup/mydb.tar.gz /var/lib/postgresql/data

tar: Removing leading `/' from member names


Ensuite, un autre conteneur PostgreSQL peut être lancé en utilisant la même base.

In [142]:
docker run -d --rm \
  --name postgres-test \
  --env POSTGRES_PASSWORD=mysecretpassword \
  -v postgres-test-data:/var/lib/postgresql/data \
  postgres:15.2

78cd390a4c7f8fa2e1996e8964a2d70dc2d6ae0c047b748f65b5c7412632319c


In [143]:
docker logs postgres-test


PostgreSQL Database directory appears to contain a database; Skipping initialization

2023-03-09 11:20:39.144 UTC [1] LOG:  starting PostgreSQL 15.2 (Debian 15.2-1.pgdg110+1) on aarch64-unknown-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit
2023-03-09 11:20:39.144 UTC [1] LOG:  listening on IPv4 address "0.0.0.0", port 5432
2023-03-09 11:20:39.144 UTC [1] LOG:  listening on IPv6 address "::", port 5432
2023-03-09 11:20:39.145 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"
2023-03-09 11:20:39.148 UTC [29] LOG:  database system was interrupted; last known up at 2023-03-09 11:20:26 UTC
2023-03-09 11:20:39.192 UTC [29] LOG:  database system was not properly shut down; automatic recovery in progress
2023-03-09 11:20:39.193 UTC [29] LOG:  invalid record length at 0/14FE140: wanted 24, got 0
2023-03-09 11:20:39.193 UTC [29] LOG:  redo is not required
2023-03-09 11:20:39.195 UTC [27] LOG:  checkpoint starting: end-of-recovery immediate wait
20

In [144]:
docker stop postgres-test

postgres-test


Il est possible de lister et de supprimer les volumes avec les commandes `ls` et `rm`. Les volumes dont les noms sont des hachés appelés volumes anonymes sont présentés plus tard. 

In [146]:
docker volume ls

DRIVER    VOLUME NAME
local     1f4f163806788759219ccfa0cdeb44d832ce7757766ea84a20ee1ac7e397e33a
local     36da887f0cd82ce378026d48cf9f629c83b1363dcffb521110b716e43de4bd09
local     759dab63ce06f80a2b542fca70c4bda0b58b292506a68ed0f25d2a69c9ce94df
local     postgres-test-data


In [147]:
docker volume rm postgres-test-data

postgres-test-data


### Réseau

Chaque conteneur dipose d'une carte réseau virtuelle et d'au moins une adresse IP. Ils peuvent appartenir à un ou plusieurs réseau virtuels et si aucun n'est précisé ils appartiennent à un réseau par défaut. Comme les conteneurs sont éphémères et que les adresses IP changent, elles ne sont généralement pas utilisées directement. 

In [157]:
docker run --quiet --rm nicolaka/netshoot ip addr show 

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: tunl0@NONE: <NOARP> mtu 1480 qdisc noop state DOWN group default qlen 1000
    link/ipip 0.0.0.0 brd 0.0.0.0
3: ip6tnl0@NONE: <NOARP> mtu 1452 qdisc noop state DOWN group default qlen 1000
    link/tunnel6 :: brd :: permaddr ee71:c6d0:82a1::
97: eth0@if98: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 02:42:ac:11:00:02 brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.2/16 brd 172.17.255.255 scope global eth0
       valid_lft forever preferred_lft forever


#### entre conteneurs

Pour que deux conteneurs puissent communiquer, il suffit qu'ils appartiennent à un même réseau et qu'ils soient nommés. Le nom du conteneur peut alors être utilisé comme un nom d'hôte sur le réseau.

Les réseaux peuvent être créés, listés et détruits avec les commandes `network {create, ls, rm}`.

In [151]:
docker network create mynet

5e36367498cd151fac5e00f5525d647644cfe701a96e7e89f6eb9ab68802f687


On peut alors exécuter un conteneur nommé `nginx-01` (il fournit un serveur web nginx) sur ce réseau avec l'option `--network`.

In [160]:
docker run --rm -d --name nginx-01 --network mynet nginx

docker: Error response from daemon: Conflict. The container name "/nginx-01" is already in use by container "7b3a041efe1de8eda3aa7f233ace02d514a2507ebddc34d908f75cad3b03c378". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


ce serveur peut être atteint depuis un autre conteneur sur le même réseau en utilisant son nom. Ici on exécute la commande `pandoc` qui convertit un site web en texte :

In [164]:
 docker run --network mynet --rm pandoc/core --to plain http://nginx-01

Welcome to nginx!

If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.

For online documentation and support please refer to nginx.org.
Commercial support is available at nginx.com.

Thank you for using nginx.


In [165]:
docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
7c502d512020   bridge    bridge    local
c159decad2bd   host      host      local
5e36367498cd   mynet     bridge    local
55fd093a79df   none      null      local


In [166]:
docker stop nginx-01
docker network rm mynet

nginx-01
mynet


si aucun réseau n'est précisé, le conteneur se trouvent dans le réseau par défaut. Il est alors obligatoire d'utiliser l'option `--link` qui est dépréciée. 

#### entre l'hôte et les conteneurs

Il est possible de mettre en place une redirection de ports entre l'hôte et les conteneur avec l'option `-p <port hote>:<port conteneur>` à la création.

In [32]:
docker run --rm -d --name nginx-01 -p 8081:80 nginx
docker run --rm -d --name nginx-02 -p 8082:80 nginx

docker stop nginx-01 nginx-02

bb4df4b1d7e7bfd7a369565daf4cb86f6970bc70def5d0e933da4ced0da273bd
014731b3219a645872a2f553b38aee6862c1f8d4903961e209eae243eb480610
nginx-01
nginx-02


### Illustration avec une base de données relationnelles

Pour illustrer les concepts précédents, nous allons créer une base de données Postgresql à partir de l'[image officielle](https://hub.docker.com/_/postgres) dans un sous-réseau `backnet` et dont les données seront persistées dans un volume nommé `postgres-01-data`.

In [168]:
docker network create backnet

50b8445835d8871ce9d6e604bce02a1f55c3ddd4918b16637cbb9a16e00addb9


In [169]:
docker run -d --rm \
  --name postgres-01 \
  --network backnet \
  --env POSTGRES_USER=dba \
  --env POSTGRES_PASSWORD=mysecretpassword \
  --env POSTGRES_DB=mydb \
  -v postgres-01-data:/var/lib/postgresql/data \
  postgres:15.2

ef2450b46e7694753ed3f504bfd98b14146c686f95d73aad76a24a65a1557d3e


Il est ensuite possible d'interroger cette base de données avec la commande `psql` exécutée dans un autre conteneur ephémère. 

In [171]:
sleep 2 

docker run --rm \
    --network backnet \
    --env PGPASSWORD=mysecretpassword \
    postgres:15.2 \
        psql -h postgres-01 -U dba mydb -c \
            "CREATE TABLE IF NOT EXISTS PERSON(ID serial PRIMARY KEY,NAME VARCHAR NOT NULL);
            INSERT INTO PERSON (NAME) VALUES ('Pierre');
            INSERT INTO PERSON (NAME) VALUES ('Marie');"

CREATE TABLE
INSERT 0 1
INSERT 0 1


Le conteneur de la base de donnée peut être détruit (cf --rm) et une autre créé avec le même volume donc sans perte de données. 

In [173]:
docker stop postgres-01

docker run -d --rm \
  --name postgres-01 \
  --network backnet \
  --env POSTGRES_USER=dba \
  --env POSTGRES_PASSWORD=mysecretpassword \
  --env POSTGRES_DB=mydb \
  -v postgres-01-data:/var/lib/postgresql/data \
  postgres:15.2

postgres-01
aaf670709ca34dfc04d44c9bf20038a69563709c4d0faafc74e402ac57774bba


In [174]:
sleep 2 

docker run --rm \
    --network backnet \
    --env PGPASSWORD=mysecretpassword \
    postgres:15.2 \
        psql -h postgres-01 -U dba mydb -c \
            "SELECT * FROM PERSON;"

 id |  name  
----+--------
  1 | Pierre
  2 | Marie
(2 rows)



Pour finir, le conteneur postgres-01 est arrêté et donc détruit, ainsi que le réseau.\
Dans notre exemple, le volume postgres-01-data est aussi détruit mais ***attention les données perdues***. 

In [176]:
docker stop postgres-01
docker network rm backnet
docker volume rm postgres-01-data

postgres-01
backnet
postgres-01-data
